In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
from modules import Encoder, Decoder
from loss import LPIPSWithDiscriminator
from distribution import DiagonalGaussianDistribution

In [26]:
class TAutoencoderKL(pl.LightningModule):
    def __init__(self,
                 latent_dim = None, 
                 *, 
                 image_size=32, 
                 patch_size=4,
                 in_channels=3, 
                 hidden_size=1152, 
                 depth=12, 
                 num_heads=6, 
                 mlp_ratio=4.0, 
                 num_classes=10, 
                 dropout_prob=0.1,
                 z_channels=None,
                 ):
        super().__init__()
        if latent_dim is None:
            self.latent_dim = (patch_size, patch_size, in_channels)
        else:
            self.latent_dim = latent_dim
        
        if z_channels is None:
            self.z_channels = self.latent_dim[2]
        else:
            self.z_channels = z_channels
        self.image_size = image_size
        assert isinstance(latent_dim, tuple) and len(latent_dim) == 3, 'Latent_dim must be a tuple of length 3 in the form (H, W, C)'
        #self.image_key = image_key
        self.encoder = Encoder(self.latent_dim, image_size=image_size, patch_size=patch_size, in_channels=in_channels, hidden_size=hidden_size, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, num_classes=num_classes, dropout_prob=dropout_prob)
        self.decoder = Decoder(self.latent_dim, image_size=image_size, patch_size=patch_size, in_channels=in_channels, hidden_size=hidden_size, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, num_classes=num_classes, dropout_prob=dropout_prob)
        self.loss = LPIPSWithDiscriminator()

        self.embed_dim = self.latent_dim[2]
        self.quant_conv = torch.nn.Conv2d(2*self.z_channels, 2*self.embed_dim, 1)
        self.post_quant_conv = torch.nn.Conv2d(self.embed_dim, self.z_channels, 1)
        
        

    def encode(self, x, y):
        h = self.encoder(x, y)
        moments = self.quant_conv(h)
        posterior = DiagonalGaussianDistribution(moments)
        return posterior

    def decode(self, z, y):
        z = self.post_quant_conv(z)
        dec = self.decoder(z, y)
        return dec

    def forward(self, input, label, sample_posterior=True):
        posterior = self.encode(input, label)
        if sample_posterior:
            z = posterior.sample()
        else:
            z = posterior.mode()
        dec = self.decode(z, label)
        return dec, posterior

    def get_input(self, x):
        if len(x.shape) == 3:
            x = x[..., None]
        if x.shape[-1] != self.image_size:
            x = x.permute(0, 3, 1, 2).to(memory_format=torch.contiguous_format).float()
        else:
            x = x.to(memory_format=torch.contiguous_format).float()
        return x

    def training_step(self, x, y, optimizer_idx):
        inputs = self.get_input(x)
        reconstructions, posterior = self(inputs, y)

        if optimizer_idx == 0:
            # train encoder+decoder+logvar
            aeloss, log_dict_ae = self.loss(inputs, reconstructions, posterior, optimizer_idx, self.global_step,
                                            last_layer=self.get_last_layer(), split="train")
            
            self.log_dict(log_dict_ae, prog_bar=False, logger=True, on_step=True, on_epoch=False)
            return aeloss

        if optimizer_idx == 1:
            # train the discriminator
            discloss, log_dict_disc = self.loss(inputs, reconstructions, posterior, optimizer_idx, self.global_step,
                                                last_layer=self.get_last_layer(), split="train")

            
            self.log_dict(log_dict_disc, prog_bar=False, logger=True, on_step=True, on_epoch=False)
            return discloss

    def validation_step(self, x, y):
        inputs = self.get_input(x)
        reconstructions, posterior = self(inputs, y)
        aeloss, log_dict_ae = self.loss(inputs, reconstructions, posterior, 0, self.global_step,
                                        last_layer=self.get_last_layer(), split="val")

        discloss, log_dict_disc = self.loss(inputs, reconstructions, posterior, 1, self.global_step,
                                            last_layer=self.get_last_layer(), split="val")

        
        self.log_dict(log_dict_ae)
        self.log_dict(log_dict_disc)
        return self.log_dict

    def configure_optimizers(self):
        lr = self.learning_rate
        opt_ae = torch.optim.Adam(list(self.encoder.parameters())+
                                  list(self.decoder.parameters())+
                                  list(self.quant_conv.parameters())+
                                  list(self.post_quant_conv.parameters()),
                                  lr=lr, betas=(0.5, 0.9))
        opt_disc = torch.optim.Adam(self.loss.discriminator.parameters(),
                                    lr=lr, betas=(0.5, 0.9))
        return opt_ae, opt_disc

    def get_last_layer(self):
        return self.decoder.final_layer.linear.weight


In [27]:
latent_dim = (16,16,3)
vae = TAutoencoderKL(latent_dim=latent_dim)

/Users/nickit/miniforge3/envs/prj/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nickit/miniforge3/envs/prj/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth


In [28]:
# randomly generate a tensor of (1,3,32,32)
a = torch.randn(1,3,32,32)
# randomly generate a label tensor of type long
y = torch.randint(0,10,(1,)).long()
b = vae(a, y)

In [29]:
b[0].shape

torch.Size([1, 3, 32, 32])

In [30]:
vae.training_step(a, y, 0)

/Users/nickit/miniforge3/envs/prj/lib/python3.11/site-packages/pytorch_lightning/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor(5142.1221, grad_fn=<AddBackward0>)